In [432]:
# Starting paths for dynamically getting all paths.
# We can fix the first two vertex of the path, because if we picked a different 
# position for the second vertex it would have rotational symmetry to the path starting with (0,0),(1,0).
# This will reduce the number of paths to check.
L1 = [[[{(0,0)},set(),(0,0)]],[[{(0,0),(1,0)},set(),(0,0),(1,0)]]]
E1 = [set(frozenset()),set(frozenset())]

# Returns one of the four possible coordinates for the next vertex on the path, num selects between them.
def NextTuple(path, num): 
    return [(path[-1][0]+1, path[-1][1]), (path[-1][0], path[-1][1]+1), (path[-1][0]-1, path[-1][1]), (path[-1][0], path[-1][1]-1)][num]

def NewEdges(path,E):
    edges = set()
    newEdgeSet = set()
    if (path[-1][0]+1, path[-1][1]) in path[0] and (path[-1][0]+1, path[-1][1]) != path[-2]: 
        edges.add((path.index((path[-1][0]+1, path[-1][1]))-1, len(path)-2)) 
    if (path[-1][0], path[-1][1]+1) in path[0] and (path[-1][0], path[-1][1]+1) != path[-2]: 
        edges.add((path.index((path[-1][0], path[-1][1]+1))-1, len(path)-2)) 
    if (path[-1][0]-1, path[-1][1]) in path[0] and (path[-1][0]-1, path[-1][1]) != path[-2]: 
        edges.add((path.index((path[-1][0]-1, path[-1][1]))-1, len(path)-2)) 
    if (path[-1][0], path[-1][1]-1) in path[0] and (path[-1][0], path[-1][1]-1) != path[-2]: 
        edges.add((path.index((path[-1][0], path[-1][1]-1))-1, len(path)-2)) 
    newEdgeSet = path[1] | edges
    #if frozenset(newEdgeSet) not in E[len(path)-3]: 
    #add = True
    #for edgeset in E[len(path)-3]:
    #    if frozenset(newEdgeSet) < edgeset: add = False
    #if add: E[len(path)-3].add(frozenset(newEdgeSet))
    
    E[len(path)-3].add(frozenset(newEdgeSet))
    #if newEdgeSet != set():
        #newEdgeSet = set('@') # @ symbol is indicating that it is a duplicate 
    #    newEdgeSet = set()
    return [path[0] | {path[-1]}] + [newEdgeSet] + path[2:]

def TotalNumContacts(path, protein):
    contacts = sum([int(protein[i-1]) * int(protein[i]) for i in range(1, len(protein))])
    contacts += sum([int(protein[edge[0]-1]) * int(protein[edge[1]-1]) for edge in path[1]])
    #print(protein, contacts)
    return contacts

def TotalNumContacts_Edge(edgeset, protein):
    contacts = sum([int(protein[i-1]) * int(protein[i]) for i in range(1, len(protein))])
    contacts += sum([int(protein[edge[0]-1]) * int(protein[edge[1]-1]) for edge in edgeset])
    #print(protein, contacts)
    return contacts

# Dynamically generates a list of all possible paths.
# Currently the filter is filtering out too many paths.
# If the filter is commented out it will work correctly.
# I am working on figuring out a better filter.
def GetPaths(L, E, N): 
    while len(L) <= N: 
        L.append([L[-1][j]+[NextTuple(L[-1][j], i)] for j in range(len(L[-1])) for i in range(4) if NextTuple(L[-1][j], i) not in L[-1][j][0]])
        E.append(set(frozenset()))
        L[-1] = [NewEdges(path,E) for path in L[-1]]
        L[-1] = list(filter(lambda x: "@" not in x[1], L[-1]))

# Simple way to display a path.        
def DisplayPath(path):  
    MaxX = max(path[0], key = lambda x: x[0])[0]
    MaxY = max(path[0], key = lambda x: x[1])[1]
    MinX = min(path[0], key = lambda x: x[0])[0]
    MinY = min(path[0], key = lambda x: x[1])[1]
    
    display = [ " ".join([str(path.index((X,Y))) if (X,Y) in path[0] else "-" for X in range(MinX, MaxX+1)]) for Y in range(MinY, MaxY+1)]
    
    for item in display: print(item)

In [433]:
%%time

N = 15

GetPaths(L1,E1,N-1)
print(len(L1[N-1]))

#for edgeset in E1: print(edgeset)
# All leading and trailing 0's can be removed from the protein,
# and then you can think of it as a protein of a shorter length. Reducing the amount of computation.
#for N in range(5,14):
proteins = [bin(i)[2:].zfill(N) for i in range(2**N)]

#average = sum([max([TotalNumContacts(path,protein) for path in L1[len(protein)-1] if path[1] != set()] + [0]) for protein in proteins])
#average = sum([max([TotalNumContacts_Edge(edgeset,protein) for edgeset in E1[len(protein)-1]]) for protein in proteins])

#maxcontactlist =[max([TotalNumContacts(path,protein) for path in L1[len(protein)-1]]) for protein in proteins]
print("Answer for N =",N, "is", average, average/(2**N))
#print(maxcontactlist)
#for i in range(2**N):
#    print(proteins[i],maxcontactlist[i])
#for path in L1[N-1]:
#    print()
#    DisplayPath(path)

593611
Answer for N = 15 is 10748 0.3280029296875
Wall time: 16.1 s


In [403]:
N = 4
for path in L1[N-1]:
    print(path)

for edgeset in E1[N-1]:
    print(edgeset)

[{(3, 0), (2, 0), (1, 0), (0, 0)}, set(), (0, 0), (1, 0), (2, 0), (3, 0)]
[{(2, 0), (1, 0), (0, 0), (2, 1)}, set(), (0, 0), (1, 0), (2, 0), (2, 1)]
[{(2, 0), (1, 0), (0, 0), (2, -1)}, set(), (0, 0), (1, 0), (2, 0), (2, -1)]
[{(1, 0), (0, 0), (1, 1), (2, 1)}, set(), (0, 0), (1, 0), (1, 1), (2, 1)]
[{(1, 2), (1, 0), (0, 0), (1, 1)}, set(), (0, 0), (1, 0), (1, 1), (1, 2)]
[{(0, 1), (1, 0), (0, 0), (1, 1)}, {(1, 4)}, (0, 0), (1, 0), (1, 1), (0, 1)]
[{(1, 0), (0, 0), (1, -1), (2, -1)}, set(), (0, 0), (1, 0), (1, -1), (2, -1)]
[{(0, -1), (1, 0), (0, 0), (1, -1)}, {(1, 4)}, (0, 0), (1, 0), (1, -1), (0, -1)]
[{(1, 0), (0, 0), (1, -1), (1, -2)}, set(), (0, 0), (1, 0), (1, -1), (1, -2)]
frozenset({(1, 4)})
frozenset()


In [435]:
print(len(E1[14]))
for s in E1: print(len(s))

12495
0
0
1
2
3
8
14
41
78
212
424
1113
2309
5953
12495


In [469]:
#N = 5
for N in range(2,13):
    bestpathset = set()
    for protein in proteins:
        best = 0
        bestpath = L1[N-1][0]
        for path in sorted(L1[N-1]):
            if TotalNumContacts(path,protein) >= best: 
                best = TotalNumContacts(path,protein)
                bestpath = path
        bestpathset.add(frozenset(bestpath[1]))

    removeset = set()
    for path1 in bestpathset:
        for path2 in bestpathset:
            if path2 < path1: removeset.add(path2)
    for path in removeset:
        bestpathset.remove(path)
        
    print()
    print(N, len(bestpathset))
    
    possible = set()
    for path in bestpathset:
        possible |= set(list(path))
    print(possible)
    
    #for path in sorted(list(bestpathset)): print(set(list(path)))
        


2 1
set()

3 1
set()

4 1
{(1, 4)}

5 2
{(2, 5), (1, 4)}

6 3
{(1, 4), (2, 5), (1, 6), (3, 6)}

7 7
{(2, 7), (4, 7), (1, 6), (3, 6), (2, 5), (1, 4)}

8 14
{(2, 7), (4, 7), (1, 4), (3, 8), (1, 8), (1, 6), (3, 6), (2, 5), (5, 8)}

9 28
{(2, 7), (6, 9), (4, 7), (4, 9), (2, 9), (1, 4), (3, 8), (1, 8), (1, 6), (3, 6), (2, 5), (5, 8)}

10 53
{(2, 7), (6, 9), (4, 7), (4, 9), (5, 10), (2, 9), (3, 10), (1, 4), (3, 8), (1, 8), (1, 6), (3, 6), (7, 10), (1, 10), (2, 5), (5, 8)}

11 99
{(6, 9), (4, 7), (6, 11), (8, 11), (1, 4), (1, 6), (2, 5), (5, 8), (4, 11), (2, 7), (5, 10), (4, 9), (2, 9), (3, 10), (2, 11), (3, 8), (1, 8), (3, 6), (7, 10), (1, 10)}


IndexError: string index out of range

In [468]:
possible = set()
for path in bestpathset:
    possible |= set(list(path))
print(possible)

{(3, 8), (4, 7), (5, 8), (1, 8)}


In [475]:
import math
N = 8
math.factorial(N) / (math.factorial(N-(N//2)) * math.factorial(N//2))

70.0

In [482]:
arr = [list(range(9))]*8
print(arr)
j = join.([arri for arri in arr])
print()
print(j)

SyntaxError: invalid syntax (<ipython-input-482-9555df29d649>, line 3)